In [46]:
:dep ndarray = "0.15"
:dep plotpy = "1.13"
:dep plotters = { version = "^0.3.5", default-features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep num-traits = "0.2"

use ndarray::prelude::{Array1,Array2}
use plotters::prelude::*;

Error: file not found for module `initialize`

Error: file not found for module `source_functions`

Error: file not found for module `utils`

Error: file not found for module `physics`

Error: file not found for module `diagnostics`

In [3]:
use ndarray::{ArrayBase, Data, Dimension, Array, array};
use num_traits::Float;

trait ArrayExtend<T,D> {
    fn min_val(&self) -> T;
    fn max_val(&self) -> T;
    fn abs_val(&self) -> Array<T,D>;
    fn exp_val(&self) -> Array<T,D>;
}

impl<S, D, T> ArrayExtend<T,D> for ArrayBase<S, D>
where
    S: Data<Elem = T>,
    D: Dimension,
    T: Float+ Copy //+ num_traits::Signed,
{
    fn min_val(&self) -> T {
        self.iter()
            .copied()
            .reduce(|a, b| if a < b { a } else { b })
            .expect("called min_val on an empty array!")
    }

    fn max_val(&self) -> T {
        self.iter()
            .copied()
            .reduce(|a, b| if a > b { a } else { b })
            .expect("called max_val on an empty array!")
    }

    fn abs_val(&self) -> Array<T,D>{
        self.mapv(|x| x.abs())
    }

    fn exp_val(&self) -> Array<T,D>{
        self.mapv(|x| x.exp())
    }
}




In [4]:
let arr = array![[3.0, 7.0, -2.0], [5.0, 0.0, 1.0]];
println!("min = {:?}", &arr.min_val()); // prints -2
println!("max = {:?}", &arr.max_val()); // prints -2
println!("abs = {:?}", &arr.abs_val()); // prints -2

min = -2.0
max = 7.0
abs = [[3.0, 7.0, 2.0],
 [5.0, 0.0, 1.0]], shape=[2, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2


In [5]:
let fmax: f64 = 20.0;
let fmin: f64 = 0.1;
let om: usize = 50;
let pi  = std::f64::consts::PI;


In [6]:
fn set_frequency(fmin: f64, fmax: f64, om: usize) -> (Array1<f64>, f64) {
    let dlnf: f64  = (fmax.ln() - fmin.ln())/ (om-1) as f64;

    let aux = Array1::from_shape_fn(om, |o| {
        (fmin.ln() + o as f64 *dlnf).exp()
    });
    (aux,dlnf)
}

let (f0, dlnf) = set_frequency(fmin, fmax, om);


In [7]:
println!("{},{},{},{}", f0[0], f0[49], f0.len(), f0[1] / f0[0])

0.10000000000000002,19.999999999999996,50,1.1141913841958424


()

In [8]:

evcxr_figure((240, 240), |root| {
    let mut chart = ChartBuilder::on(&root)
        // .caption("My Plot", ("sans-serif", 30).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0..f0.len(),
                            // *f0.iter().min_by(|a,b| a.partial_cmp(b).unwrap()).unwrap() as f64 ..
                            f0.min_val() as f64..f0.max_val() as f64)?;

    chart.configure_mesh().draw()?;

    // Plot the Array1
    chart.draw_series(LineSeries::new(
        f0.iter().enumerate().map(|(i, &y)| (i, y as f64)),
        &RED,
    ))?;

    Ok(())
})

2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
16.0
 
 
 
18.0
 
 
 
 
0
 
 
 
5
 
 
 
10
 
 
 
15
 
 
 
20
 
 
 
25
 
 
 
30
 
 
 
35
 
 
 
40
 
 
 
45
 
 
 
50

In [9]:
let nx  : usize = 11;
let istart : usize = 0;
let iend   : usize = nx;
let x0 = ndarray::Array1::<f64>::linspace(0.,nx as f64 -1.,nx);

In [10]:
let mut f = ndarray::Array2::<f64>::zeros((nx,om));
let mut x = ndarray::Array2::<f64>::zeros((nx,om));

for i in 0..nx{
    for o in 0..om{
            f[[i,o]] = f0[o];
            x[[i,o]] = x0[i];
        }
    }

()

In [11]:
let grav_accel: f64 = 9.8;
let surface_tension: f64 = 0.074;
let water_density: f64 = 1e3;
let water_depth: f64 = 1e3;

In [12]:
fn wavenumber(istart: usize,
    iend  : usize,
    om    : usize,
    frequency: &Array2<f64>,
    water_depth: f64,
    grav_accel: f64,
    surface_tension: f64,
    water_density: f64) -> Array2<f64> {
    
    let pi  = std::f64::consts::PI;
    
    
    let frequency_nondim = 2. * pi * frequency * (water_depth/grav_accel).sqrt();
    let mut k: Array2<f64> = frequency_nondim.mapv(|x| x.powi(2));
    let surface_tension_nondim = surface_tension / (grav_accel * water_density * water_depth.powi(2));

    for i in istart..iend{
        for o in 0..om{
        let mut count: usize = 0;
        let mut dk: f64 = 0.;
    
            // while dk.abs()>tol {
            loop {
                let t = (k[[i,o]]).tanh();
                
                dk = -(frequency_nondim[[i,o]].powi(2) -
                       k[[i,o]] * t * (1.+ surface_tension_nondim * (k[[i,o]]).powi(2)))
                       / ( 3. * surface_tension_nondim * (k[[i,o]]).powi(2) * t
                       + t
                       + k[[i,o]] * (1.+surface_tension_nondim*(k[[i,o]]).powi(2)) 
                       * (1.-t.powi(2)));
                k[[i,o]] -= dk;
                
                count += 1;    
                if count >= 1000 {
                    break;  // Escape if stuck
                }
                }
            k[[i,o]] = k[[i,o]]/water_depth;
            }
        }
    k
}

In [13]:
use plotters::prelude::*;
use ndarray::{Array2,array,s};


evcxr_figure((240, 240), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(10)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0.1f64..0.5f64, 0f64..2.0f64)?;

    chart.configure_mesh().draw()?;

    let k = wavenumber(istart, iend, om, &f, 32., grav_accel, surface_tension, water_density);
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (f[[0,j]], k[[0,j]])),
        &BLUE,
    ))?;

    let k = wavenumber(istart, iend, om, &f, 2., grav_accel, surface_tension, water_density);
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (f[[0,j]], k[[0,j]])),
        &GREEN,
    ))?;

    let k = wavenumber(istart, iend, om, &f, 1., grav_accel, surface_tension, water_density);
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (f[[0,j]], k[[0,j]])),
        &RED,
    ))?;


    
    
    // chart.draw_series(LineSeries::new(
    //     (0..f.shape()[1]).map(|j| (f[[4,j]], cg[[0,j]])),
    //     &RED,
    // ))?;

    
    Ok(())
}).style("width:100%")



0.0
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
0.8
 
 
 
1.0
 
 
 
1.2
 
 
 
1.4
 
 
 
1.6
 
 
 
1.8
 
 
 
2.0
 
 
 
 
0.1
 
 
 
0.15
 
 
 
0.2
 
 
 
0.25
 
 
 
0.3
 
 
 
0.35
 
 
 
0.4
 
 
 
0.45
 
 
 
0.5

In [14]:
let k = wavenumber(istart, iend, om, &f, 32., grav_accel, surface_tension, water_density);

In [15]:
let depth = Array2::<f64>::ones((nx,om)) * water_depth;

In [16]:
fn phase_speed(frequency: &Array2<f64>,wavenumber: &Array2<f64>) -> Array2<f64> {
    let pi  = std::f64::consts::PI;
    2. *pi * frequency/wavenumber
}

In [17]:
fn group_speed(surface_tension: f64, frequency: &Array2<f64>, wavenumber: &Array2<f64>, water_depth: &Array2<f64>, grav_accel: f64,
water_density: f64) -> Array2<f64> {
    // let pi  = std::f64::consts::PI;
    let cp = phase_speed(&frequency, &wavenumber);
    let kd = wavenumber * water_depth;
    let sigma_k2 = surface_tension * wavenumber.mapv(|x| x.powi(2));
    cp * (0.5 + &kd / &kd.mapv(|x| x.sinh()) + &sigma_k2/ (water_density * grav_accel * &sigma_k2))
    
}

In [18]:
let k = wavenumber(istart, iend, om, &f, water_depth, grav_accel, surface_tension, water_density);
let cp =phase_speed( &f, &k);
let cg =group_speed(surface_tension, &f, &k, &depth, grav_accel, water_density);
let dk = 2.*pi* &f/&cg;

println!("{},{}", cg[[0,0]], cg[[0,49]])

7.800183856516162,0.11990281538031511


()

In [19]:
use plotters::prelude::*;
use ndarray::{Array2,array,s};


evcxr_figure((240, 240), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0f64..5f64, 0f64..10f64)?;

    chart.configure_mesh().draw()?;


    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (f[[0,j]], cp[[0,j]])),
        &BLUE,
    ))?;

    
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (f[[4,j]], cg[[0,j]])),
        &RED,
    ))?;

    
    Ok(())
}).style("width:100%")


0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
9.0
 
 
 
10.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0

In [20]:

evcxr_figure((360, 360), |root| {
    let mut chart = ChartBuilder::on(&root)
        // .caption("My Plot", ("sans-serif", 30).into_font())
        .margin(10)
        .x_label_area_size(60)
        .y_label_area_size(30)
        .build_cartesian_2d(0f64..20f64, 0f64..1000f64)?;

    chart.configure_mesh().draw()?;

    
    chart.draw_series(LineSeries::new(
        (0..dk.shape()[1]).map(|j| (f[[0,j]], dk[[0,j]])),
        &RED,
    ))?;

    Ok(())
})

0.0
 
 
 
100.0
 
 
 
200.0
 
 
 
300.0
 
 
 
400.0
 
 
 
500.0
 
 
 
600.0
 
 
 
700.0
 
 
 
800.0
 
 
 
900.0
 
 
 
1000.0
 
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
16.0
 
 
 
18.0
 
 
 
20.0

In [21]:
fn source_input(
    wind_speed: f64,
    frequency: &Array2<f64>,
    wavenumber: &Array2<f64>,
    phase_speed:&Array2<f64>,
) -> Array2<f64> {

    let sheltering_coefficient: f64 = 0.11;
    let air_density: f64 = 1.2;
    let water_density: f64 = 1e3;
    let current: f64 = 0.;
    let grav_accel: f64 = 9.8;
    let pi  = std::f64::consts::PI;


    let wind_speed_relative = wind_speed - phase_speed - current;
    let mut s_in = sheltering_coefficient * &wind_speed_relative * &wind_speed_relative.abs_val()
                   * frequency * wavenumber ;
    s_in = s_in * air_density / water_density / grav_accel * 2. * pi ;
    s_in
}



In [22]:
use plotters::coord::types::RangedCoordf64;


evcxr_figure((480, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(10)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d((0.001..15.).log_scale(),
                             (1e-10..1e2).log_scale())?;

    chart.configure_mesh().draw()?;
    
    let ssin = source_input(10., &f, &k, &cp);
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (f[[0,j]], ssin[[0,j]])),
        &BLUE,
    ))?;

    let ssin = source_input(6., &f, &k, &cp);
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (f[[0,j]], ssin[[0,j]])),
        &GREEN,
    ))?;

    let ssin = source_input(2., &f, &k, &cp);
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (f[[0,j]], ssin[[0,j]])),
        &RED,
    ))?;
    
    Ok(())
}).style("width:100%")

1e-10
 
 
 
1e-9
 
 
 
1e-8
 
 
 
1e-7
 
 
 
1e-6
 
 
 
9.999999999999999e-6
 
 
 
9.999999999999999e-5
 
 
 
0.001
 
 
 
0.01
 
 
 
0.1
 
 
 
1.0
 
 
 
10.0
 
 
 
100.0
 
 
 
 
0.01
 
 
 
0.1
 
 
 
1.0
 
 
 
10.0

In [23]:
use ndarray::{Array2, Axis};

/// Computes sum(spectrum_k * k^2 * dk, axis=1) for 2D k and dk
fn weighted_sum_2d(spectrum_k: &Array2<f64>, k: &Array2<f64>, dk: &Array2<f64>) -> ndarray::Array1<f64> {
    let weighted = spectrum_k * &k.mapv(|x| x.powi(2)) * dk;
    weighted.sum_axis(Axis(1)) // sum along columns (axis=1)
}


fn cumsum_axis1(arr: &Array2<f64>) -> Array2<f64> {
    let mut out = arr.clone();
    for mut row in out.axis_iter_mut(Axis(0)) { // iterate over rows
        let mut sum = 0.0;
        for elem in row.iter_mut() {
            sum += *elem;
            *elem = sum;
        }
    } 
    out
}

fn weighted_cumsum(spectrum_k: &Array2<f64>, k: &Array2<f64>, dk: &Array2<f64>) -> Array2<f64> {
    // Elementwise multiply
    let weighted = spectrum_k * k.mapv(|x| x.powi(2)) * dk;

    // Cumulative sum along columns (axis=1)
    cumsum_axis1(&weighted)
}

In [24]:
fn mean_squared_slope(spectrum_k: &Array2<f64>, k: &Array2<f64>, dk: &Array2<f64>) -> Array1<f64> {
    weighted_sum_2d(&spectrum_k, &k, &dk)
}

In [25]:
fn mean_squared_slope_long(spectrum_k: &Array2<f64>, k: &Array2<f64>, dk: &Array2<f64>) -> Array2<f64> {
    weighted_cumsum(&spectrum_k, &k, &dk)
}

In [26]:
let Fk = Array2::<f64>::ones((k.shape()[0], k.shape()[1])) * 1e-15;

let mss_val = mean_squared_slope_long(&Fk, &k, &dk);



In [27]:
use plotters::coord::types::RangedCoordf64;


evcxr_figure((300, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(20)
        .x_label_area_size(50)
        .y_label_area_size(30)
        .build_cartesian_2d((0.001f64..20.0f64).log_scale(),
                             0.0..0.000001f64)?;


    chart.configure_mesh().draw()?;
    
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (f[[0,j]], mss_val[[0,j]])),
        &BLUE,
    ))?;

    Ok(())
}).style("width:100%")

0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
 
0.01
 
 
 
0.1
 
 
 
1.0
 
 
 
10.0

In [28]:
fn saturation_spectrum(spectrum_k: &Array2<f64>, k: &Array2<f64>) -> Array2<f64> {
    spectrum_k * k.mapv(|x| x.powi(4))
}

In [29]:
use plotters::coord::types::RangedCoordf64;

let Sk_temp = saturation_spectrum(&Fk, &k);

evcxr_figure((400, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(20)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(0.001f64..510.0f64,
                             0.0..0.00009f64)?;


    chart.configure_mesh().draw()?;
    
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (k[[0,j]], Sk_temp[[0,j]])),
        &BLUE,
    ))?;

    Ok(())
}).style("width:100%")

0.0
 
 
 
0.00001
 
 
 
0.00002
 
 
 
0.00003
 
 
 
0.00004
 
 
 
0.00005
 
 
 
0.00006
 
 
 
0.00007
 
 
 
0.00008
 
 
 
0.0001
 
 
 
 
50.0
 
 
 
100.0
 
 
 
150.0
 
 
 
200.0
 
 
 
250.0
 
 
 
300.0
 
 
 
350.0
 
 
 
400.0
 
 
 
450.0
 
 
 
500.0

In [30]:
fn source_dissipation(spectrum_k: &Array2<f64>,
                      frequency: &Array2<f64>,
                      k: &Array2<f64>,
                      dk: &Array2<f64>) -> Array2<f64> {
    let pi  = std::f64::consts::PI;

    let dissipation_coefficient: f64 =42.;
    let dissipation_power: f64 =2.4;
    let mss_coefficient: f64 =120.;

    let omega = 2.*pi*frequency;
    
    let mut mss = Array2::<f64>::zeros((k.shape()[0], k.shape()[1]));
    if mss_coefficient >0. {
        mss = mean_squared_slope_long(&spectrum_k, &k, &dk);
    }
    else {
        mss = Array2::<f64>::zeros((k.shape()[0], k.shape()[1]));
    }
    let mss_effect = (1. + mss_coefficient * &mss).mapv(|x| x.powi(2));

    let B_k = saturation_spectrum(&spectrum_k, &k);    
    dissipation_coefficient * omega * mss_effect * B_k.map(|x| x.powf(dissipation_power))
    
}


In [31]:
fn wind_wave_balance(source_input: &Array2<f64>,
                     frequency: &Array2<f64>,
                     wavenumber: &Array2<f64>) -> Array2<f64> {
let pi  = std::f64::consts::PI;

let dissipation_coefficient: f64 = 42.;
let dissipation_power: f64 = 2.4;
let mss: f64 = 0.;
let mss_coefficient: f64 = 120.;

let omega = 2. * pi * frequency;
let mss_effect = (1. + mss_coefficient*mss).powi(2); 
let mut Fk = (source_input 
          /(omega * dissipation_coefficient * mss_effect))
          .mapv(|x| x.powf(1. / dissipation_power))
          / wavenumber.mapv(|x| x.powi(4));

for val in Fk.iter_mut() {
    if val.is_nan() {
        *val = 0.0;
    }
}

Fk  
}

In [32]:
let Fk = wind_wave_balance(&source_input(10., &f, &k, &cp), &f, &k);


In [33]:
use plotters::coord::types::RangedCoordf64;


evcxr_figure((300, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(20)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d((0.001f64..510.0f64).log_scale(),
                             (1e-11..10f64).log_scale())?;


    chart.configure_mesh().draw()?;
    
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (k[[0,j]], Fk[[0,j]])),
        &BLUE,
    ))?;

    Ok(())
}).style("width:100%")

1e-11
 
 
 
9.999999999999999e-11
 
 
 
9.999999999999999e-10
 
 
 
9.999999999999999e-9
 
 
 
9.999999999999998e-8
 
 
 
9.999999999999997e-7
 
 
 
9.999999999999997e-6
 
 
 
9.999999999999998e-5
 
 
 
0.0009999999999999998
 
 
 
0.009999999999999998
 
 
 
0.09999999999999998
 
 
 
0.9999999999999998
 
 
 
9.999999999999998
 
 
 
 
0.01
 
 
 
0.1
 
 
 
1.0
 
 
 
10.0
 
 
 
100.0

In [34]:
use plotters::coord::types::RangedCoordf64;
let mss_val = mean_squared_slope_long(&Fk, &k, &dk);

evcxr_figure((300, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(20)
        .x_label_area_size(50)
        .y_label_area_size(30)
        .build_cartesian_2d((0.01f64..600.0f64).log_scale(),
                             0.0..0.45f64)?;


    chart.configure_mesh().draw()?;
    
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (k[[0,j]], mss_val[[0,j]])),
        &BLUE,
    ))?;

    Ok(())
}).style("width:100%")

//TODO! understand why the results are different from Milan's

0.0
 
 
 
0.05
 
 
 
0.1
 
 
 
0.15
 
 
 
0.2
 
 
 
0.25
 
 
 
0.3
 
 
 
0.35
 
 
 
0.4
 
 
 
0.45
 
 
 
 
0.1
 
 
 
1.0
 
 
 
10.0
 
 
 
100.0

In [35]:
fn source_wave_interaction(istart: usize,
                           iend  : usize,
                           om    : usize,
                           spectrum_k: &Array2<f64>,
                           k: &Array2<f64>,
                           dk: &Array2<f64>) -> Array2<f64> {
    let snl_coefficient: f64=1.;
    let mut snl = Array2::<f64>::zeros((k.shape()[0], k.shape()[1]));

    for o in 0..om-1 {
        for i in istart..iend {
            snl[[i,o]] = snl_coefficient * (spectrum_k[[i,o+1]] - spectrum_k[[i,o]])/ dk[[i,o+1]];
        }
    }
    snl
}
    

In [36]:
use plotters::coord::types::RangedCoordf64;
let snl_val = source_wave_interaction(istart, iend, om, &Fk, &k, &dk);

evcxr_figure((300, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(20)
        .x_label_area_size(50)
        .y_label_area_size(30)
        .build_cartesian_2d((0.01f64..600.0f64).log_scale(),
                             -5.0f64..16f64)?;


    chart.configure_mesh().draw()?;
    
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (k[[0,j]], snl_val[[0,j]])),
        &BLUE,
    ))?;

    Ok(())
}).style("width:100%")

//TODO! understand why the results are different from Milan's

-4.0
 
 
 
-2.0
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
16.0
 
 
 
 
0.1
 
 
 
1.0
 
 
 
10.0
 
 
 
100.0

In [37]:
use plotters::coord::types::RangedCoordf64;
let sds_val = source_dissipation( &Fk, &f, &k, &dk);

evcxr_figure((300, 300), |root| {
    let mut chart = ChartBuilder::on(&root)
        .margin(20)
        .x_label_area_size(50)
        .y_label_area_size(30)
        .build_cartesian_2d((0.01f64..600.0f64).log_scale(),
                             (1e-6f64..1e6f64).log_scale())?;


    chart.configure_mesh().draw()?;
    
    chart.draw_series(LineSeries::new(
        (0..f.shape()[1]).map(|j| (k[[0,j]], sds_val[[0,j]])),
        &BLUE,
    ))?;

    Ok(())
}).style("width:100%")

//TODO! understand why the results are different from Milan's

1e-5
 
 
 
0.0001
 
 
 
0.001
 
 
 
0.01
 
 
 
0.1
 
 
 
1.0
 
 
 
10.0
 
 
 
100.0
 
 
 
1000.0
 
 
 
10000.0
 
 
 
100000.0
 
 
 
1000000.0
 
 
 
 
0.1
 
 
 
1.0
 
 
 
10.0
 
 
 
100.0

In [38]:
fn advect(istart: usize,
          iend  : usize,
          om    : usize,
          q: &Array2<f64>,
          cp: &Array2<f64>,
          x: &Array2<f64>,
         ) -> Array2<f64> {
    let mut res = Array2::<f64>::zeros((q.shape()[0], q.shape()[1]));

    for i in istart..iend-1 {
        for o in 0..om {
            res[[i+1, o]] = (q[[i+1,o]] - q[[i,o]])/ (x[[i+1,o]] - x[[i,o]]);
            res[[0,o]] = q[[0,o]]/x[[1,o]] - x[[0,o]];
        }
    }
    &res * cp
}

let ones = Array2::<f64>::ones((iend,om));



[[15.597184614131475, 13.998658523833774, 12.563962408478977, 11.276305654989939, 10.120618493938807, ..., 0.23214845726087815, 0.2323739201861734, 0.23382253096467512, 0.23632708731362323, 0.23975670082168704],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0]]


In [39]:
fn significant_wave_height(spectrum_k: &Array2<f64>, dk: &Array2<f64>) -> Array1<f64>{
    
    let istart: usize = 0;
    let iend  : usize = spectrum_k.shape()[0];
    let om    : usize = spectrum_k.shape()[1];
    
    let mut swh = Array1::<f64>::zeros(iend);

    for i in istart..iend {
        for o in 0..om{
            swh[[i]]+= spectrum_k[[i,o]] * dk[[i,o]];
        }
    }

    4. * swh.mapv(|x| x.sqrt())
        
}

In [40]:
use ndarray::prelude::s


fn integrate(
    istart: usize,
    iend:   usize,
    om:     usize,
    x: &Array2<f64>,
    Fk_init: &Array2<f64>,
    f: &Array2<f64>,
    k: &Array2<f64>,
    cp: &Array2<f64>,
    cg: &Array2<f64>,
    wspd: f64,
    duration: f64,
    output_interval: f64,
    mss_coefficient: f64,
    snl_coefficient: f64,
    exp_growth_factor: f64) -> Array2<f64> {
    
    let num_time_steps:usize = (duration/output_interval) as usize;
    let num_grid_points = k.shape()[0];
    let exp_growth_factor: f64 = 0.1;
    let pi  = std::f64::consts::PI;

    let dk = 2. * pi * f/cg;
    
    let mut Fk = Fk_init.clone();
    let mut swh = Array2::<f64>::zeros((num_time_steps, iend));
    
    let mut count: usize = 0;
    for n in 0..num_time_steps {
        let mut elapsed: f64 = 0.;

        // while count < 4 {
        while elapsed < output_interval {
            let mut Sin = source_input(wspd, &f, &k, &cp);
            let mut Sds = source_dissipation(&Fk, &f, &k, &dk);
            let mut Snl = source_wave_interaction(istart, iend, om, &Fk, &k, &dk);
            
            let mut aux = &Sin -&Sds;
            
            
            let dt = (exp_growth_factor / &aux.abs_val().max_val())
                .min(output_interval - elapsed);

            Fk = &Fk * (dt * &aux).mapv(|x| x.exp()) +  dt * (&Snl - advect(istart, iend, om, &Fk, &cg, &x));
            swh.row_mut(count).assign(&significant_wave_height(&Fk, &dk));
            
            elapsed += dt;
        }
        count += 1;
    }
    swh
}
     

In [41]:
use std::time::Instant;


let Fk_init = wind_wave_balance(&source_input(0.8, &f, &k, &cp), &f, &k);
let wspd = Array1::<f64>::ones(nx) *15.;

let start = Instant::now();
let swh = integrate(
    istart,
    iend,
    om,
    &x,
    &Fk_init,
    &f,
    &k,
    &cp,
    &cg,
    15.,
    60.,
    1.,
    1.,
    42.,
    0.1);
let duration = start.elapsed();

println!("Function took: {:?}", duration);
println!("Function took: {} ms", duration.as_millis());
println!("Function took: {} μs", duration.as_micros());

Function took: 300.448441ms
Function took: 300 ms
Function took: 300448 μs


In [42]:
use plotters::prelude::*;
use ndarray::Array2;



let figure = evcxr_figure((600, 600), |root| {
    let (rows, cols) = swh.dim();
    
    root.fill(&WHITE)?;
    
    let mut chart = ChartBuilder::on(&root)
        .caption("Pcolor Plot", ("sans-serif", 30))
        .margin(10)
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..(cols as f64), 0f64..(rows as f64))?;
    
    chart.configure_mesh().draw()?;
    
    let min_val = swh.min_val();
    let max_val = swh.max_val();
    
    for i in 0..rows {
        for j in 0..cols {
            let value = swh[[i, j]];
            let normalized = if max_val != min_val {
                (value - min_val) / (max_val - min_val)
            } else {
                0.5
            };
            
            // Better turbo colormap approximation (purple → blue → cyan → green → yellow → orange → red)
            let (red, green, blue) = if normalized < 0.15 {
                // Purple to dark blue
                let t = normalized / 0.15;
                ((48.0 + t * (30.0 - 48.0)) as u8, (18.0 + t * (4.0 - 18.0)) as u8, (59.0 + t * (157.0 - 59.0)) as u8)
            } else if normalized < 0.35 {
                // Dark blue to cyan
                let t = (normalized - 0.15) / 0.2;
                ((30.0 + t * (0.0 - 30.0)) as u8, (4.0 + t * (181.0 - 4.0)) as u8, (157.0 + t * (255.0 - 157.0)) as u8)
            } else if normalized < 0.6 {
                // Cyan to green
                let t = (normalized - 0.35) / 0.25;
                (0, ((181.0 + t * (255.0 - 181.0)) as u8), ((255.0 + t * (30.0 - 255.0)) as u8))
            } else if normalized < 0.8 {
                // Green to yellow
                let t = (normalized - 0.6) / 0.2;
                (((t * 255.0) as u8), 255, ((30.0 + t * (0.0 - 30.0)) as u8))
            } else {
                // Yellow to red
                let t = (normalized - 0.8) / 0.2;
                (255, ((255.0 + t * (30.0 - 255.0)) as u8), 0)
            };
            let color = RGBColor(red, green, blue);
            
            chart.draw_series(std::iter::once(Rectangle::new([
                (j as f64, (rows - i - 1) as f64),
                ((j + 1) as f64, (rows - i) as f64)
            ], color.filled())))?;
        }
    }
    
    Ok(())
});

figure

Pcolor Plot
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
10.0
 
 
 
20.0
 
 
 
30.0
 
 
 
40.0
 
 
 
50.0
 
 
 
60.0
 
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0